In [1]:
%matplotlib inline

import itertools
import os

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.spatial.distance import cosine
import seaborn as sns

In [2]:
df=pd.read_table('../sample_input_files/changeo_combined.samples.tsv')

In [8]:
metadata=pd.read_csv('../sample_input_files/metadata_csv.txt').rename(columns = {'filename':'Sample'})

In [9]:
merged_df=pd.merge(df,metadata,on='Sample')

In [10]:
merged_df

,Sample,0,1,2,3,4,5,6,7,8,...,59430,59431,59432,59433,patient_id,tissue_id,subset_1,subset_2,replicate,dna
0,D233-LLN-CD8-TEM-rep4-200ng_S46,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D233,LLN,CD8,TEM,rep4,200ng_S46
1,D233-BM-CD8-TEM-rep4-200ng_S38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D233,BM,CD8,TEM,rep4,200ng_S38
2,D299-BM-CD8-TRM-rep2-41ng_S72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D299,BM,CD8,TRM,rep2,41ng_S72
3,D255-LLN-CD8-TRM-rep1-46ng_S47,307.0,439.0,4.0,0.0,0.0,48.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D255,LLN,CD8,TRM,rep1,46ng_S47
4,D280-LLN-CD8-TEM-rep2-26ng_S76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D280,LLN,CD8,TEM,rep2,26ng_S76
5,D233-BM-CD8-TEM-rep3-200ng_S27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D233,BM,CD8,TEM,rep3,200ng_S27
6,D255-BM-CD8-TRM-rep2-46ng_S57,105.0,101.0,216.0,0.0,0.0,286.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D255,BM,CD8,TRM,rep2,46ng_S57
7,D287-BM-CD8-TEM-rep4-55ng_S38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D287,BM,CD8,TEM,rep4,55ng_S38
8,D255-BM-CD8-TEM-rep2-46ng_S58,3629.0,105.0,633.0,2.0,0.0,10.0,3.0,3.0,1.0,...,0.0,0.0,0.0,0.0,D255,BM,CD8,TEM,rep2,46ng_S58
9,D280-LLN-CD8-TEM-rep1-26ng_S86,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,D280,LLN,CD8,TEM,rep1,26ng_S86


In [5]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output, Image

In [6]:
merged_df=merged_df.head()

In [8]:
items

['All', ' D233', ' D255', ' D280', ' D299']

In [11]:
items = ['All']+sorted(merged_df['patient_id'].unique().tolist())
 
def view(x=''):
    if x=='All': return merged_df
    return merged_df[merged_df['patient_id']==x]
 
w = widgets.Select(options=items)
v=interactive(view, x=w)
display(v)

interactive(children=(Select(description='x', options=('All', 'D233', 'D255', 'D280', 'D287', 'D299'), value='…

In [ ]:
res_df = {}
    for l1, l2 in itertools.combinations(df.index, 2):
        res_df.setdefault(l2, {})[l1] = res_df.setdefault(l1, {})[l2] = 1 - cosine(
            list(df.loc[l1].values),
            list(df.loc[l2].values),
        )
        
data=pd.DataFrame(res_df)